In [1]:
import cv2

In [2]:
img1_path = r"C:\Users\cbamm\Downloads\legal1.png"
img1 = cv2.imread(img1_path)
scale = 0.1  # adjust 0.3–0.5 as needed
h, w = img1.shape[:2]
img1 = cv2.resize(img1, (int(w*scale), int(h*scale)))

cv2.imshow("Legal Text", img1)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [3]:
door_path = r"C:\Users\cbamm\Downloads\frame_0000.png"
door = cv2.imread(door_path)
door = cv2.resize(door, (int(door.shape[1]*0.4), int(door.shape[0]*0.4)))  # adjust scale
print("Door size (H,W):", door.shape[:2])

Door size (H,W): (768, 432)


In [4]:
cv2.imshow("Door", door)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
import numpy as np

# decal corners (full rectangle)
src_pts = np.float32([[0, 0], [img1.shape[1], 0],
                      [img1.shape[1], img1.shape[0]], [0, img1.shape[0]]])

# destination points (choose on truck door)
dst_pts = np.float32([
    [120, 450],   # top-left
    [320, 440],   # top-right
    [330, 600],   # bottom-right
    [110, 610]    # bottom-left
])


In [6]:
M = cv2.getPerspectiveTransform(src_pts, dst_pts)
warped = cv2.warpPerspective(img1, M, (door.shape[1], door.shape[0]))

cv2.imshow("Warped decal", warped)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [7]:
# --- Overlay warped decal onto door ---
overlay = door.copy()

# Create mask from warped decal (non-black pixels)
gray_mask = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
_, mask = cv2.threshold(gray_mask, 1, 255, cv2.THRESH_BINARY)

# Extract decal and background
warped_fg = cv2.bitwise_and(warped, warped, mask=mask)
bg = cv2.bitwise_and(overlay, overlay, mask=cv2.bitwise_not(mask))

# Combine them
final = cv2.add(bg, warped_fg)

cv2.imshow("Projected Decal on Door", final)
cv2.waitKey(0)
cv2.destroyAllWindows()